In [2]:
import pandas as pd
import requests
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import yfinance as yfin
import pandas_ta as ta
import datetime as dt

In [3]:
data = pd.read_csv("GBPUSD_H1_202201030000_202401040000.csv")
all_rows = []
for i, row in data.iterrows():
    new_row = str(row.values[0]).split("\t")
    new_row = new_row[:-2]
    new_row = (new_row[0] + " " + new_row[1], new_row[2:])
    all_rows.append(new_row)
    
all_dates = [val[0] for val in all_rows]
all_OHLCV = [val[1] for val in all_rows]
data = pd.DataFrame(data = all_OHLCV, index = all_dates, columns = ["Open", "High", "Low", "Close", "Volume"])
data = data.astype(float)
data.index = pd.to_datetime(data.index)

data

,Open,High,Low,Close,Volume
2022-01-03 00:00:00,1.35237,1.35340,1.34310,1.34787,41257.0
2022-01-04 00:00:00,1.34723,1.35575,1.34593,1.35278,45941.0
2022-01-05 00:00:00,1.35274,1.35983,1.35225,1.35557,48604.0
2022-01-06 00:00:00,1.35540,1.35590,1.34904,1.35354,53573.0
2022-01-07 00:00:00,1.35278,1.35973,1.35271,1.35910,45412.0
...,...,...,...,...,...
2024-01-03 20:00:00,1.26447,1.26546,1.26408,1.26463,3756.0
2024-01-03 21:00:00,1.26466,1.26764,1.26307,1.26639,8729.0
2024-01-03 22:00:00,1.26638,1.26671,1.26541,1.26637,5351.0
2024-01-03 23:00:00,1.26638,1.26685,1.26616,1.26635,1372.0


In [4]:
data = data.join(data.ta.macd())
data["ATR"] = data.ta.atr(length = 14)
data

,Open,High,Low,Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,ATR
2022-01-03 00:00:00,1.35237,1.35340,1.34310,1.34787,41257.0,NaN,NaN,NaN,NaN
2022-01-04 00:00:00,1.34723,1.35575,1.34593,1.35278,45941.0,NaN,NaN,NaN,NaN
2022-01-05 00:00:00,1.35274,1.35983,1.35225,1.35557,48604.0,NaN,NaN,NaN,NaN
2022-01-06 00:00:00,1.35540,1.35590,1.34904,1.35354,53573.0,NaN,NaN,NaN,NaN
2022-01-07 00:00:00,1.35278,1.35973,1.35271,1.35910,45412.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024-01-03 20:00:00,1.26447,1.26546,1.26408,1.26463,3756.0,-0.000581,0.000444,-0.001025,0.001785
2024-01-03 21:00:00,1.26466,1.26764,1.26307,1.26639,8729.0,-0.000317,0.000566,-0.000883,0.001984
2024-01-03 22:00:00,1.26638,1.26671,1.26541,1.26637,5351.0,-0.000108,0.000620,-0.000728,0.001935
2024-01-03 23:00:00,1.26638,1.26685,1.26616,1.26635,1372.0,0.000055,0.000627,-0.000572,0.001846


In [5]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

class MyMACD(Strategy):
    
    tp_ratio = 4
    sl_ratio = 1
    
    fast = 14
    slow = 24
    signal = 7
    
    rsi = 14
    
    def init(self):
        super().init()
        self.MACD, self.MACDh, self.MACDs = self.I(ta.macd, pd.Series(self.data.Close), self.fast, self.slow, self.signal)
        self.RSI = self.I(ta.rsi, pd.Series(self.data.Close), self.rsi)
        
    def next(self):
        super().next()
        sl_dist = self.sl_ratio * self.data.ATR[-1]
        
        if len(self.trades) > 0:
            if self.trades[-1].is_long and self.RSI[-1] >= 90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.RSI[-1] <= 10:
                self.trades[-1].close()
        elif crossover(self.MACD, self.MACDs) and len(self.trades) == 0:
            if self.position.is_short:
                self.position.close()
            sl = self.data.Close[-1] - sl_dist
            tp = self.data.Close[-1] + (self.tp_ratio * sl_dist)
            self.buy(sl = sl, tp = tp, size = 0.2)
        elif crossover(self.MACDs, self.MACD) and len(self.trades) == 0:
            if self.position.is_long:
                self.position.close()
            sl = self.data.Close[-1] + sl_dist
            tp = self.data.Close[-1] +- (self.tp_ratio * sl_dist)
            self.sell(sl = sl, tp = tp, size = 0.2)
            
bt = Backtest(data, MyMACD, cash=20000)
stat = bt.run()
stat

C:\Users\edgar\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Start                     2022-01-03 00:00:00
End                       2024-01-04 00:00:00
Duration                    731 days 00:00:00
Exposure Time [%]                   57.200811
Equity Final [$]                 20429.162723
Equity Peak [$]                  20653.658773
Return [%]                           2.145814
Buy & Hold Return [%]               -6.054738
Return (Ann.) [%]                     1.03221
Volatility (Ann.) [%]                1.568217
Sharpe Ratio                         0.658206
Sortino Ratio                        1.058528
Calmar Ratio                         0.701374
Max. Drawdown [%]                   -1.471697
Avg. Drawdown [%]                   -0.296011
Max. Drawdown Duration      145 days 22:00:00
Avg. Drawdown Duration       21 days 07:00:00
# Trades                                  293
Win Rate [%]                        18.771331
Best Trade [%]                       6.929815
Worst Trade [%]                     -1.857246
Avg. Trade [%]                    

In [6]:
stats, hm = bt.optimize(fast = range(5, 15),
                    slow = range(15, 25),
                    signal = range(5, 15),
                    rsi = range(3, 7),
                    maximize = "Return [%]",
                   return_heatmap = True)

C:\Users\edgar\anaconda3\Lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 4000 configurations.
  output = _optimize_grid()


  0%|          | 0/14 [00:00<?, ?it/s]

In [7]:
stats

Start                     2022-01-03 00:00:00
End                       2024-01-04 00:00:00
Duration                    731 days 00:00:00
Exposure Time [%]                   52.454361
Equity Final [$]                 20831.239771
Equity Peak [$]                  20831.272731
Return [%]                           4.156199
Buy & Hold Return [%]               -6.054738
Return (Ann.) [%]                    1.989164
Volatility (Ann.) [%]                 1.53897
Sharpe Ratio                         1.292529
Sortino Ratio                        2.244681
Calmar Ratio                         1.587785
Max. Drawdown [%]                   -1.252791
Avg. Drawdown [%]                    -0.13601
Max. Drawdown Duration      102 days 00:00:00
Avg. Drawdown Duration        9 days 06:00:00
# Trades                                  306
Win Rate [%]                        30.392157
Best Trade [%]                       6.929815
Worst Trade [%]                     -1.857246
Avg. Trade [%]                    

In [8]:
hm.sort_values().iloc[-3:]

fast  slow  signal  rsi
13    17    10      4      4.144483
11    17    12      4      4.156199
12    17    11      4      4.156199
Name: Return [%], dtype: float64